In [1]:
pip install nltk scikit-learn streamlit


  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached pyarrow-19.0.1-cp311-cp311-win_amd64.whl.metadata (3.4 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached attrs-25.1.0-py3-none-any.whl.metadata (10 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-a

In [2]:
import nltk
import random
import os
import ssl
import streamlit as st

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [4]:
ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
nltk.data.path.append(os.path.abspath('nltk_data'))
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ALPHA\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [6]:
intents = [
    # Basic Greetings & Conversations
    {
        "tag": "greeting",
        "patterns": ["Hi", "Hello", "Hey", "How are you?", "What's up?", "Yo!", "Howdy!", "Good morning", "Good evening"],
        "responses": ["Hi there!", "Hello!", "Hey!", "I'm good, how about you?", "Nothing much, just here to chat!", "Howdy, partner!", "Good morning! Hope you have a great day!"]
    },
    {
        "tag": "goodbye",
        "patterns": ["Bye", "See you later", "Goodbye", "Take care", "Catch you later", "Adios", "Peace out"],
        "responses": ["Goodbye!", "See you later!", "Take care!", "Catch you soon!", "Stay safe!", "Until next time!"]
    },
    {
        "tag": "thanks",
        "patterns": ["Thank you", "Thanks", "Thanks a lot", "I appreciate it", "Many thanks"],
        "responses": ["You're welcome!", "No problem!", "Glad I could help!", "Anytime!", "Always here for you!"]
    },

    # Fun & Curious Topics
    {
        "tag": "space",
        "patterns": ["Tell me about space", "What is a black hole?", "How big is the universe?", "Are aliens real?"],
        "responses": [
            "Space is vast and mostly empty, but filled with mysteries like black holes and distant galaxies!",
            "A black hole is a region in space where gravity is so strong that nothing, not even light, can escape.",
            "The observable universe is about 93 billion light-years across and expanding!",
            "Aliens? We haven't found them yet, but the universe is too big for us to be alone, don't you think?"
        ]
    },
    {
        "tag": "technology",
        "patterns": ["What’s the latest in AI?", "Tell me about quantum computing", "How does blockchain work?"],
        "responses": [
            "AI is evolving fast! From ChatGPT to self-driving cars, it's shaping the future.",
            "Quantum computing is a new way of processing information using quantum bits, allowing for massive parallelism.",
            "Blockchain is a decentralized ledger technology that secures transactions across networks."
        ]
    },
    {
        "tag": "history",
        "patterns": ["Tell me a historical fact", "Who was the greatest leader in history?", "What was the most important invention?"],
        "responses": [
            "Did you know that ancient Romans used concrete, and some of their structures still stand today?",
            "Many consider leaders like Alexander the Great, Napoleon, or Abraham Lincoln as among the greatest.",
            "The printing press revolutionized knowledge sharing, but electricity and the internet changed everything too!"
        ]
    },

    # Life & Advice
    {
        "tag": "motivation",
        "patterns": ["Motivate me", "Give me a motivational quote", "I need inspiration"],
        "responses": [
            "Success is not final, failure is not fatal: It is the courage to continue that counts.",
            "Your only limit is your mind. Keep pushing forward!",
            "Hard work beats talent when talent doesn’t work hard."
        ]
    },
    {
        "tag": "life_advice",
        "patterns": ["What’s the best life advice?", "How can I be successful?", "How do I stay happy?"],
        "responses": [
            "Focus on progress, not perfection. Learn, adapt, and keep going!",
            "Success is built on discipline, patience, and continuous learning.",
            "Happiness comes from gratitude, meaningful relationships, and enjoying the present moment."
        ]
    },
    {
        "tag": "productivity",
        "patterns": ["How do I stay productive?", "What are the best productivity tips?", "How can I stop procrastinating?"],
        "responses": [
            "Break tasks into small steps, set deadlines, and stay consistent.",
            "Use the Pomodoro technique: 25-minute work sessions with short breaks in between.",
            "Eliminate distractions, set priorities, and create a daily to-do list!"
        ]
    },

    # Professional & Career
    {
        "tag": "job_search",
        "patterns": ["How do I get a job?", "Best way to prepare for an interview?", "What skills should I learn for the future?"],
        "responses": [
            "Build a strong resume, apply strategically, and network with industry professionals.",
            "For interviews, research the company, practice common questions, and showcase your problem-solving skills.",
            "AI, data science, cloud computing, and cybersecurity are future-proof skills worth learning!"
        ]
    },
    {
        "tag": "leadership",
        "patterns": ["How can I be a great leader?", "What are leadership qualities?", "Tell me leadership tips"],
        "responses": [
            "Great leaders listen, inspire, and lead by example.",
            "Communication, empathy, decision-making, and vision are key leadership qualities.",
            "Leadership isn’t about control—it’s about empowering others to succeed."
        ]
    },
    {
        "tag": "work_life_balance",
        "patterns": ["How do I balance work and life?", "Tips for avoiding burnout?", "How can I manage stress at work?"],
        "responses": [
            "Set boundaries, prioritize self-care, and unplug from work when needed.",
            "Burnout happens when you don’t rest. Take breaks and do activities you enjoy.",
            "Time management and saying ‘no’ to unnecessary tasks help in reducing stress."
        ]
    },

    # Fun & Casual
    {
        "tag": "jokes",
        "patterns": ["Tell me a joke", "Make me laugh", "Say something funny"],
        "responses": [
            "Why do programmers prefer dark mode? Because light attracts bugs!",
            "I told my wife she should embrace her mistakes. She gave me a hug!",
            "Why don’t skeletons fight each other? They don’t have the guts!"
        ]
    },
    {
        "tag": "random_facts",
        "patterns": ["Tell me a random fact", "Give me an interesting fact", "Surprise me"],
        "responses": [
            "Octopuses have three hearts and blue blood!",
            "Honey never spoils. Archaeologists have found pots of honey in ancient tombs that are still edible!",
            "Bananas are berries, but strawberries aren’t!"
        ]
    },
    {
        "tag": "dreams",
        "patterns": ["Why do we dream?", "What do dreams mean?", "Can you interpret my dream?"],
        "responses": [
            "Dreams are your brain's way of processing information and emotions.",
            "There are many theories, but dreams might reflect your thoughts, fears, or desires.",
            "I’m not a dream interpreter, but keeping a dream journal might help you find patterns!"
        ]
    },

    # Science & Health
    {
        "tag": "health_tips",
        "patterns": ["How can I stay healthy?", "Give me fitness tips", "What are good eating habits?"],
        "responses": [
            "Stay active, eat balanced meals, and get enough sleep!",
            "Exercise regularly and drink plenty of water.",
            "Avoid processed foods, eat more fruits and veggies, and listen to your body."
        ]
    },
    {
        "tag": "mental_health",
        "patterns": ["How do I reduce stress?", "What are ways to improve mental health?", "How do I handle anxiety?"],
        "responses": [
            "Practice mindfulness, meditate, and take deep breaths.",
            "Spend time with loved ones, engage in hobbies, and seek support when needed.",
            "Anxiety can be overwhelming, but grounding techniques and self-care help a lot."
        ]
    },

    # Miscellaneous
    {
        "tag": "music",
        "patterns": ["What’s a good song to listen to?", "Tell me about music", "Who is the greatest musician?"],
        "responses": [
            "Music is a universal language that connects people emotionally.",
            "Depends on your taste! Try some classic rock, jazz, or lo-fi beats.",
            "Many consider Beethoven, Michael Jackson, or The Beatles as legendary artists!"
        ]
    },
    {
        "tag": "books",
        "patterns": ["Recommend me a book", "What’s the best book?", "Tell me about famous books"],
        "responses": [
            "If you like self-improvement, try *Atomic Habits* by James Clear.",
            "For fiction, *1984* by George Orwell is a must-read.",
            "Many love *The Alchemist*, *Harry Potter*, and *Sapiens*!"
        ]
    },
    {
        "tag": "ai_future",
        "patterns": ["What is the future of AI?", "Will AI take over?", "How will AI evolve?"],
        "responses": [
            "AI is expected to revolutionize industries, from healthcare to finance!",
            "AI won't take over, but it will change how we work and live.",
            "The future of AI is exciting—think about self-driving cars, advanced robotics, and creative AI!"
        ]
    },
    {
        "tag": "space_exploration",
        "patterns": ["Is space travel possible?", "Tell me about Mars missions", "When will humans go to Mars?"],
        "responses": [
            "Space travel is becoming a reality with companies like SpaceX and NASA pushing boundaries.",
            "Mars missions are planned for the 2030s! Scientists are already testing technology.",
            "Humanity is closer than ever to becoming an interplanetary species!"
        ]
    },
    {
        "tag": "black_holes",
        "patterns": ["What is a black hole?", "Can we survive a black hole?", "Tell me about singularities"],
        "responses": [
            "A black hole is a region in space where gravity is so strong that nothing can escape—not even light!",
            "Falling into a black hole would stretch you infinitely—scientists call it 'spaghettification.'",
            "The singularity at the center of a black hole is still a mystery to physics!"
        ]
    },
    {
        "tag": "quantum_computing",
        "patterns": ["What is quantum computing?", "How does a quantum computer work?", "Can quantum computers break encryption?"],
        "responses": [
            "Quantum computing uses qubits that can be both 0 and 1 at the same time—super powerful!",
            "Unlike regular computers, quantum computers perform complex calculations at lightning speed!",
            "Yes, quantum computers might break today's encryption, but post-quantum cryptography is being developed!"
        ]
    },
    {
        "tag": "multiverse",
        "patterns": ["Is the multiverse real?", "What are parallel universes?", "Can we travel to another universe?"],
        "responses": [
            "The multiverse theory suggests that infinite universes might exist!",
            "Some scientists believe parallel universes could exist but proving it is a challenge.",
            "If we ever discover a way to travel between universes, it would be the biggest breakthrough in science!"
        ]
    },
    {
        "tag": "ancient_civilizations",
        "patterns": ["What happened to Atlantis?", "Were aliens involved in ancient structures?", "Tell me about lost civilizations"],
        "responses": [
            "Atlantis is a legendary lost city—some believe it was inspired by real ancient cultures.",
            "Theories about aliens helping build structures like the pyramids are fun, but there's no solid proof!",
            "Many ancient civilizations like the Mayans and the Indus Valley were highly advanced for their time."
        ]
    },
    {
        "tag": "time_travel",
        "patterns": ["Is time travel possible?", "Can we go back in time?", "What is the grandfather paradox?"],
        "responses": [
            "Theoretically, time travel is possible using concepts like wormholes, but we don't have the technology yet.",
            "Going back in time might create paradoxes—like meeting your past self!",
            "The Grandfather Paradox suggests that if you traveled back and prevented your grandfather from meeting your grandmother, you might never be born!"
        ]
    },
    {
        "tag": "simulation_theory",
        "patterns": ["Are we living in a simulation?", "What is the simulation theory?", "Can we escape the simulation?"],
        "responses": [
            "Some scientists and philosophers suggest we might be in a super-advanced simulation!",
            "If our universe is a simulation, we'd need to find 'glitches' to prove it!",
            "Escaping a simulation—if it exists—would be like breaking the laws of reality itself!"
        ]
    },
    {
        "tag": "dark_matter",
        "patterns": ["What is dark matter?", "Can we see dark matter?", "How does dark matter work?"],
        "responses": [
            "Dark matter makes up most of the universe, but we can't see it—only its effects on gravity!",
            "Scientists are still trying to understand what dark matter is made of.",
            "Without dark matter, galaxies would fly apart! It plays a crucial role in the cosmos."
        ]
    },
    {
        "tag": "extraterrestrial_life",
        "patterns": ["Do aliens exist?", "Have we been visited by aliens?", "What are UFOs?"],
        "responses": [
            "The universe is vast—many scientists believe alien life is out there, but we haven't found proof yet.",
            "UFOs (now called UAPs) are unidentified aerial phenomena—some might be advanced tech, others still unexplained!",
            "If we find alien life, it could be microbial, intelligent, or something beyond our imagination!"
        ]
    }

]
